In [1]:
#hide earrings 
import warnings
warnings.filterwarnings('ignore')
import numpy as np 
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

In [3]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest, chi2

In [4]:
df = pd.read_csv('train (1).csv')

In [5]:
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [6]:
df.drop(columns=['Ticket', 'PassengerId', 'Cabin','Name'],axis=1, inplace=True)

In [7]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

#### plain or steps for pipeline

In [8]:
#Separate Feature and Target Matrix
def get_seprate_features(df_id,col_id):
    #col_id is special col or target col b/c this the whole model is going to be created 
    global X, y
    X = df_id.drop(col_id,axis = 1) 
    y = df_id[col_id]
get_seprate_features(df,'Survived')

In [9]:
X.head(10)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S
5,3,male,NaN,0,0,8.4583,Q
6,1,male,54.0,0,0,51.8625,S
7,3,male,2.0,3,1,21.0750,S
8,3,female,27.0,0,2,11.1333,S
9,2,female,14.0,1,0,30.0708,C


In [10]:
y.sample(2)

507    1
785    0
Name: Survived, dtype: int64

In [11]:
y.head(6)

0    0
1    1
2    1
3    1
4    0
5    0
Name: Survived, dtype: int64

In [12]:
def get_train_test_split(array1, array2, test_size, random_state):
    global X_train, X_test, y_train, y_test
    X_train, X_test, y_train, y_test = train_test_split(array1, array2, test_size=test_size,random_state=random_state)
get_train_test_split(X,y,0.2,42)

In [13]:
X_train.head(3)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.500,S
733,2,male,23.0,0,0,13.000,S
382,3,male,32.0,0,0,7.925,S


In [14]:
y_train.sample(5)

188    0
222    0
239    0
265    0
372    0
Name: Survived, dtype: int64

In [15]:
# 1st step imputation transfer. 

# here we call ColumnTransformer function, And
# you pass list of tuples (which are 2 in is this case). 

# (1) ('Impute_age', SimpleImputer(),[2]) mean (Impute_age is the name of tuple) then
# (it fills the missing values by SimpleImputer() == mean_value of that particular column). And
# (we give index of column in which we want imputation)

#(2) ('Impute_embarked', SimpleImputer(strategy='most_frequent'),[6])then
# (it fills the missing values by SimpleImputer(strategy='most_frequent') == most_frequent_value of that particular column). And
# (we give index of column in which we want imputation)

# remainder='passthrough' (it tells the ColumnTransformer() that we want to keep the remaining columns as is it)
trf1 = ColumnTransformer(
    [('Impute_age', SimpleImputer(),[2]),
     ('Impute_embarked', SimpleImputer(strategy='most_frequent'),[6])
    ], remainder='passthrough'
)

In [16]:
df['Age'].mean()

29.69911764705882

In [17]:
df['Embarked'].mode()

0    S
Name: Embarked, dtype: object

In [18]:
trf1

ColumnTransformer(remainder='passthrough',
                  transformers=[('Impute_age', SimpleImputer(), [2]),
                                ('Impute_embarked',
                                 SimpleImputer(strategy='most_frequent'),
                                 [6])])

In [19]:
# step 2 OneHotEcode (for categorical columns as per of dataset)
# we can do OneHotEncoder of multiple column in single code of line b/c everything is going be done inside data
trf2 = ColumnTransformer(
    [('ohe_sex_embarked',OneHotEncoder(sparse=False, handle_unknown='ignore'),[1,6])
    ],remainder = "passthrough")
# from the code will get 5 eXtra column which are (2 from Sex) and (3 from Embarked)
# so from now we have 10 total columns

In [20]:
trf2

ColumnTransformer(remainder='passthrough',
                  transformers=[('ohe_sex_embarked',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse=False),
                                 [1, 6])])

In [21]:
# step (3) Scaling. (all columns have similAR numerical values)
# for features selection you must use MinMaxScaler() good result. however we can also use StandardScaler()
# MinMaxScaler() apply on all column from index (0) to index (8)
# the slice() have the range of columns and on those it apply  MinMaxScaler() respectively
trf3 = ColumnTransformer(
    [('scale',MinMaxScaler(),slice(0,10))
    ])

In [22]:
trf3

ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))])

In [23]:
# 4) Feature Sesection (select (the k) means most important columns)
trf4 = SelectKBest(score_func=chi2,k=8)

In [24]:
trf4

SelectKBest(k=8, score_func=<function chi2 at 0x000001DEFA2B98B0>)

In [25]:
# 5) Train model (by a particular model)
trf5 = DecisionTreeClassifier()

In [26]:
trf5

DecisionTreeClassifier()

#### create pipeline

In [27]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)])

In [28]:
pipe

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('Impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000001DEFA2B98B0>)),
                ('trf5', DecisionTreeClassifier())])

In [29]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('Impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000001DEFA2B98B0>)),
                ('trf5', DecisionTreeClassifier())])

##### Pipeline v/s make_pipeline

Pipeline requires naming of steps and make_pipeline
(same condition apply on ColumnTransformer and make_column_transformer)

#### make_pipeline

#### Explore the pipeline 

In [30]:
# we want to see the name of the steps
pipe.named_steps
# results you see the pipe is in the form dictionary 

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('Impute_age', SimpleImputer(), [2]),
                                 ('Impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(k=8, score_func=<function chi2 at 0x000001DEFA2B98B0>),
 'trf5': DecisionTreeClassifier()}

In [31]:
# we want to 9 how trf1 works
pipe.named_steps['trf1'].transformers_
# now we got a list that has 3 tuples 

[('Impute_age', SimpleImputer(), [2]),
 ('Impute_embarked', SimpleImputer(strategy='most_frequent'), [6]),
 ('remainder', 'passthrough', [0, 1, 3, 4, 5])]

In [32]:
# now I want to what is mean value of trf1's 1st tuple
# now I want take 1st item which is SimpleImputer() == mean of that column 

pipe.named_steps['trf1'].transformers_[0][1].statistics_
# the result shows the mean value

array([29.49884615])

In [33]:
pipe.named_steps['trf1'].transformers_[1][1].statistics_

array(['S'], dtype=object)

In [34]:
# it very helpful in debugging 

In [35]:
from sklearn import set_config
set_config(display='diagram')

In [36]:
y_pred = pipe.predict(X_test)

In [37]:
from sklearn.metrics import accuracy_score

In [38]:
accuracy_score(y_test,y_pred)

0.6256983240223464

#### cross_validation using pipeline

In [39]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean()
# cv ==bcross_val_score
# you can take cross_val_score by using pipe,
# it train and split dataset at 5 different times and every time run the algorithm, ccalculate the mean score of accuracy 

0.6391214419383433

#### GridSearch using pipeline

In [40]:
params = {
    'trf5__max_depth':[1,2,3,4,5,None]
}

In [41]:
from sklearn.model_selection import GridSearchCV

In [42]:
grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('Impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('Impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x000001DEFA2B98B0>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [43]:
grid.best_score_

0.6391214419383433

In [44]:
grid.best_params_

{'trf5__max_depth': 2}

In [45]:
import pickle
#here we doing encoding
pickle.dump(pipe,open('pipe.pkl','wb'))
#these are binary file you can not see however you can use it 